In [2]:
# !pip install MTCFeatures

In [3]:
import MTCFeatures
from MTCFeatures import MTCFeatureLoader
import pandas as pd

In [4]:
# MTCFeatures.downloadData(dest='user')

In [5]:
fl = MTCFeatureLoader('MTC-FS-INST-2.0')
seqs = fl.sequences()

In [6]:
%%time
phrase_data = []
for ii, x in enumerate(seqs):
    phrase_data.append({
        'id': x['id'],
        **x['features']
    })

EOFError: Compressed file ended before the end-of-stream marker was reached

In [7]:
df_phrase = pd.DataFrame(phrase_data)
df_phrase.shape # (13492, 62)

(13492, 62)

In [8]:
import numpy as np

In [9]:
# Chaque ligne du set représente une mélodie, représentée par des features.

print(np.array(df_phrase['pitch'][0]).shape) # pitch représente la hauteur des notes
print(np.array(df_phrase['duration'][0]).shape) # duration représente la durée des notes
print(np.array(df_phrase['phrase_end'][0]).shape) # phrase_end dit si la note est la dernière de la phrase

print(df_phrase['pitch'][0])
print(df_phrase['duration'][0])
print(df_phrase['phrase_end'][0])
# On peut donc voir que chacun de ses 3 features est une liste de la même longueur que le nombre de notes dans la mélodie.

# combien de fin de phrase dans le premier exemple ?

print(np.sum(df_phrase['phrase_end'][0])) # 4

# On pense découpere les séquences en sous séquences par notes avec un nombre fixe de notes par sous séquence. On pourra tester de diviser les séquences en sous séquences de nombre 
# fixe de note pour toutes les séquences ou bien de diviser les séquences par un nombre fixe pour toutes les séquences.

print(df_phrase['beat'][0]) # beat représente le temps de la note dans la mesure

# On peut aussi faire des sous séquences basées sur les mesures pour éviter de couper une mesure en deux.

print(df_phrase['beat_fraction_str'][0]) # beat_fraction_str représente le temps de la note dans la mesure sous forme de fraction


(44,)
(44,)
(44,)
['E4', 'C4', 'A3', 'G3', 'C4', 'C4', 'D4', 'C4', 'D4', 'E4', 'A3', 'G3', 'G3', 'F#3', 'G3', 'A3', 'F4', 'E4', 'D4', 'E4', 'D4', 'G3', 'D4', 'E4', 'D4', 'C4', 'A3', 'G3', 'E4', 'F4', 'E4', 'A3', 'E4', 'D4', 'G3', 'A3', 'G3', 'G3', 'F4', 'E4', 'D4', 'A3', 'B3', 'C4']
[1.0, 0.5, 0.5, 0.75, 0.25, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 1.5]
[False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True]
4
[1.0, 2.0, 2.5, 1.0, 1.75, 2.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5,

In [10]:
# print(df_phrase.describe()) # Pas utile pour les features qui sont des listes et trop long, ça sert à r ici

In [11]:
df_phrase.info() # 7302 sequences avec paroles

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13492 entries, 0 to 13491
Data columns (total 62 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     13492 non-null  object
 1   scaledegree            13492 non-null  object
 2   scaledegreespecifier   13492 non-null  object
 3   tonic                  13492 non-null  object
 4   mode                   13492 non-null  object
 5   metriccontour          13492 non-null  object
 6   imaweight              13492 non-null  object
 7   pitch40                13492 non-null  object
 8   midipitch              13492 non-null  object
 9   diatonicpitch          13492 non-null  object
 10  diatonicinterval       13492 non-null  object
 11  chromaticinterval      13492 non-null  object
 12  pitchproximity         13492 non-null  object
 13  pitchreversal          13492 non-null  object
 14  nextisrest             13492 non-null  object
 15  restduration_frac  

In [12]:
for col in df_phrase.columns:
    t = type(df_phrase[col][0])
    if t != list:
        print(col, t)

id <class 'str'>
lyrics <class 'float'>
noncontentword <class 'float'>
wordend <class 'float'>
phoneme <class 'float'>
rhymes <class 'float'>
rhymescontentwords <class 'float'>
wordstress <class 'float'>
melismastate <class 'float'>


7302 séquences avec des paroles

In [13]:
from tqdm import tqdm

In [14]:
def generate_subsequences(df, length=8):
    """
    Divise les colonnes de type liste d'un DataFrame en sous-séquences de taille `length`.

    Args:
        df (pd.DataFrame): Le DataFrame d'entrée contenant des colonnes avec des listes.
        length (int): La longueur des sous-séquences souhaitées.

    Returns:
        pd.DataFrame: Un nouveau DataFrame contenant les sous-séquences comme lignes.
    """
    subsequences = []

    for _, row in df.iterrows():
        min_length = min(len(row[col]) for col in df.columns)

        for start_idx in range(0, min_length, length):
            subsequence = {
                col: row[col][start_idx:start_idx + length]
                for col in df.columns
            }
            subsequences.append(subsequence)

    return pd.DataFrame(subsequences)

In [15]:
# toutes les valeurs possibles de la colonne 'timesignature' dans le DataFrame
timesignature = []
for x in df_phrase['timesignature']:
    for y in x:
        if y not in timesignature:
            timesignature.append(y)
print(timesignature)

['2/4', '4/4', '6/8', '3/4', '3/8', '5/8', '7/8', '2/2', '12/8', '8/16', None, '6/4', '3/2', '3/16', '9/8', '9/4', '4/2', '5/4', '4/8', '8/8', '10/8', '2/8', '1/4', '7/4', '22/8', '2/16', '8/4', '13/8', '19/16', '6/2', '1/2', '2/1', '10/4', '10/2', '12/2', '16/2', '5/2']


In [16]:
i = 1
j = 0

print("midipitch:", df_phrase['midipitch'][i][:])
print("chromaticinterval:", df_phrase['chromaticinterval'][i][:])
print("tonic:", df_phrase['tonic'][i][:])
print("mode:", df_phrase['mode'][i][:])

# print("scaledegree:", df_phrase['scaledegree'][i][:])
print("timesignature:", df_phrase['timesignature'][i][:])
print("beatstrength:", df_phrase['beatstrength'][i][:])
# print("metriccontour:", df_phrase['metriccontour'][i][:])
# print("imaweight:", df_phrase['imaweight'][i][:])
# print("imacontour:", df_phrase['imacontour'][i][:])
print("=====================================")
print("duration:", df_phrase['duration'][i][:])
# print("duration_frac:", df_phrase['duration_frac'][i][:])
# print("duration_fullname:", df_phrase['duration_fullname'][i][:])
print("durationcontour:", df_phrase['durationcontour'][i][:])
print("IOI:", df_phrase['IOI'][i][:])
print("IOR:", df_phrase['IOR'][i][:])
# print("onsettick:", df_phrase['onsettick'][i][:]) # onsettick représente le temps de la note en ticks mais ne sert pas dans notre cas car ne donne aucune information utiles, comme un ID

print("=====================================")
print("beatfraction:", df_phrase['beatfraction'][i][:]) # durée de la note
print("beat_str:", df_phrase['beat_str'][i][:])
print("beat_fraction_str:", df_phrase['beat_fraction_str'][i][:])
print("beat:", df_phrase['beat'][i][:]) # beat_str + beat_fraction_str
# print("songpos:", df_phrase['songpos'][i][:]) # position de la note dans la chanson similaire à onsettick

print("=====================================")
# print("beatinsong:", df_phrase['beatinsong'][i][:]) # meme chose que pour songpos et onsettick
# print("nextisrest:", df_phrase['nextisrest'][i][:])
print("restduration_frac:", df_phrase['restduration_frac'][i][:]) # plus précis que nextisrest
print("phrase_ix:", df_phrase['phrase_ix'][i][:]) #numero de la phrase, inutile (id)
print("=====================================")
print("phrasepos", df_phrase['phrasepos'][i][:]) # position de la note dans la phrase
print("phrase_end", df_phrase['phrase_end'][i][:]) # indique si la note est la fin de la phrase
print("=====================================")
print("beatinphrase", df_phrase['beatinphrase'][i][:]) 
print("beatinphrase_end", df_phrase['beatinphrase_end'][i][:])
print("=====================================")
print("melismastate", df_phrase['melismastate'][i])

midipitch: [72, 73, 74, 72, 70, 69, 65, 69, 69, 69, 69, 74, 72, 60, 62, 64, 65, 67, 69, 70, 72, 74, 76, 79, 79, 79, 79, 60, 65, 67, 69, 69, 69, 60, 65, 67, 69, 69, 69, 72, 74, 72, 67, 69, 70, 69, 69, 60, 65, 67, 69, 69, 69, 60, 65, 67, 67, 69, 72, 74, 72, 67, 67, 72, 70, 67, 65, 74, 74, 76, 77, 76, 77, 74, 72, 69, 70, 70, 72, 67, 69, 70, 67, 69, 74, 72, 74, 74, 76, 77, 76, 77, 74, 72, 69, 69, 69, 67, 74, 72, 64, 65, 72, 72, 72, 74, 72, 70, 67, 65, 69, 69, 69, 69, 69, 67, 70, 70, 70, 70, 70, 69, 72, 72, 72, 72, 72, 72, 74, 72, 70, 67, 65, 69, 69, 69, 69, 69, 67, 74, 72, 64, 65]
chromaticinterval: [None, 1, 1, -2, -2, -1, -4, 4, 0, 0, 0, 5, -2, -12, 2, 2, 1, 2, 2, 1, 2, 2, 2, 3, 0, 0, 0, -19, 5, 2, 2, 0, 0, -9, 5, 2, 2, 0, 0, 3, 2, -2, -5, 2, 1, -1, 0, -9, 5, 2, 2, 0, 0, -9, 5, 2, 0, 2, 3, 2, -2, -5, 0, 5, -2, -3, -2, 9, 0, 2, 1, -1, 1, -3, -2, -3, 1, 0, 2, -5, 2, 1, -3, 2, 5, -2, 2, 0, 2, 1, -1, 1, -3, -2, -3, 0, 0, -2, 7, -2, -8, 1, 7, 0, 0, 2, -2, -2, -3, -2, 4, 0, 0, 0, 0, -2, 3, 0, 

Les attributs phrasepos et phrase_end donnent explicitement une indication sur la fin de phrase. beatinphrase et beatinphrase_end donnent également des indications explicites sur le début et la fin des phrases. 

on remarque qu'il y a des features qui sont équivalentes comme le `pitch` et le `midipitch`. Ce dernier n'est qu'une représentation du premier en midi, c'est-à-dire en entier. On pourra donc omettre le pitch et ne garder que le midipitch car il est plus simple à manipuler.

on a C5 = 72 et C#5 = 73 car c'est la note juste après C5

On doit chercher toutes les features qui sont corrélées pour ne pas utiliser des features inutiles

[`pitch`, `midipitch`, `pitch40`, `diatonicpitch`] -> `midipitch` : hauteur de la note

[`contour3`, `contour5`, `diatonicinterval`, `chromaticinterval`] -> `chromaticinterval` : variation par rapport à la note précédente

[`tonic`] -> `tonic`

[`mode`] -> `mode`

[`scaledegree`, `scaledegreespecifier`] -> `scaledegree` : peut-être les 2 ?

[`timesignature`] -> `timesignature`

[`beatstrength`] -> `beatstrength`

[`mtriccontour`] -> `metriccontour`

[`imaweight`] -> `imaweight`

[`imacontour`] -> `imacontour`

[`duration`, `duration_frac`, `duration_fullname`] -> `duration` : durée de la note (forme de fraction et nom chaine de caractère)

[`durationcontour`] -> `durationcontour`

[`IOI`, `IOI_frac`] -> `IOI`

[`IOR`, `IOR_frac`] -> `IOR` : ratio entre IOI précédent et IOI actuel

[`beatfraction`] -> `beatfraction` : durée de la note

[`beat_fraction_str`, `beat_str`, `beat`] -> `beat` : `beat_fraction_str` + `beat_str`

[`nextisrest`, `restduration_frac`] -> `restduration_frac` : pause qui suit la note

[`phrasepos`, `phrase_end`] -> `phrase_pos` : fin de la phrase

[`beatinphrase`, `beatinphrase_end`] -> les deux sont similaires, donne info sur fin de phrase donc peut être à ne pas prendre en compte







[`onsettick`, `songpos`, `beatinsong`, `phrase_ix`] -> aucune features utile












In [17]:
features = np.array(['midipitch', 'chromaticinterval', 'tonic', 'mode', 'scaledegree', 'timesignature', 'beatstrength', 'metriccontour', 'imaweight', 'imacontour', 'duration', 'durationcontour', 'beatfraction', 'beat', 'restduration_frac', 'phrase_end'])
new_df = df_phrase[features]

sub_seqs_df = generate_subsequences(new_df, length=8)

In [18]:
sub_seqs_df.head()

,midipitch,chromaticinterval,tonic,mode,scaledegree,timesignature,beatstrength,metriccontour,imaweight,imacontour,duration,durationcontour,beatfraction,beat,restduration_frac,phrase_end
0,"[64, 60, 57, 55, 60, 60, 62, 60]","[None, -4, -3, -2, 5, 0, 2, -2]","[C, C, C, C, C, C, C, C]","[major, major, major, major, major, major, maj...","[3, 1, 6, 5, 1, 1, 2, 1]","[2/4, 2/4, 2/4, 2/4, 2/4, 2/4, 2/4, 2/4]","[1.0, 0.5, 0.25, 1.0, 0.125, 0.5, 1.0, 0.5]","[None, -, -, +, -, +, +, -]","[0.798013, 0.663907, 0.150662, 0.945364, 0.013...","[None, -, -, +, -, +, +, -]","[1.0, 0.5, 0.5, 0.75, 0.25, 1.0, 1.0, 0.5]","[None, -, =, +, -, +, =, -]","[1, 1/2, 1/2, 3/4, 1/4, 1, 1, 1/2]","[1.0, 2.0, 2.5, 1.0, 1.75, 2.0, 1.0, 2.0]","[None, None, None, None, None, None, None, None]","[False, False, False, False, False, False, Fal..."
1,"[62, 64, 57, 55, 55, 54, 55, 57]","[2, 2, -7, -2, 0, -1, 1, 2]","[C, C, C, C, C, C, C, C]","[major, major, major, major, major, major, maj...","[2, 3, 6, 5, 5, 4, 5, 6]","[2/4, 2/4, 2/4, 2/4, 2/4, 2/4, 2/4, 2/4]","[0.25, 1.0, 0.25, 0.5, 1.0, 0.5, 0.25, 1.0]","[-, +, -, +, +, -, -, +]","[0.195364, 0.812914, 0.162252, 0.627483, 0.951...","[-, +, -, +, +, -, -, +]","[0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5]","[=, =, =, +, =, -, =, =]","[1/2, 1/2, 1/2, 1, 1, 1/2, 1/2, 1/2]","[2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0]","[None, None, None, None, None, None, None, None]","[False, False, False, True, False, False, Fals..."
2,"[65, 64, 62, 64, 62, 55, 62, 64]","[8, -1, -2, 2, -2, -7, 7, 2]","[C, C, C, C, C, C, C, C]","[major, major, major, major, major, major, maj...","[4, 3, 2, 3, 2, 5, 2, 3]","[2/4, 2/4, 2/4, 2/4, 2/4, 2/4, 2/4, 2/4]","[0.25, 0.5, 1.0, 0.5, 0.25, 1.0, 1.0, 0.5]","[-, +, +, -, -, +, =, -]","[0.183775, 0.642384, 0.837748, 0.692053, 0.25,...","[-, +, +, -, -, +, -, -]","[0.5, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 0.5]","[=, +, =, -, =, +, =, -]","[1/2, 1, 1, 1/2, 1/2, 1, 1, 1/2]","[1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.0, 2.0]","[None, None, None, None, None, 1, None, None]","[False, False, False, False, False, True, Fals..."
3,"[62, 60, 57, 55, 64, 65, 64, 57]","[-2, -2, -3, -2, 9, 1, -1, -7]","[C, C, C, C, C, C, C, C]","[major, major, major, major, major, major, maj...","[2, 1, 6, 5, 3, 4, 3, 6]","[2/4, 2/4, 2/4, 2/4, 2/4, 2/4, 2/4, 2/4]","[0.25, 1.0, 0.25, 0.5, 1.0, 0.5, 0.25, 1.0]","[-, +, -, +, +, -, -, +]","[0.243377, 0.870861, 0.205298, 0.630795, 0.917...","[-, +, -, +, +, -, -, +]","[0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5]","[=, =, =, +, =, -, =, =]","[1/2, 1/2, 1/2, 1, 1, 1/2, 1/2, 1/2]","[2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0]","[None, None, None, None, None, None, None, None]","[False, False, False, False, False, False, Fal..."
4,"[64, 62, 55, 57, 55, 55, 65, 64]","[7, -2, -7, 2, -2, 0, 10, -1]","[C, C, C, C, C, C, C, C]","[major, major, major, major, major, major, maj...","[3, 2, 5, 6, 5, 5, 4, 3]","[2/4, 2/4, 2/4, 2/4, 2/4, 2/4, 2/4, 2/4]","[0.25, 0.5, 1.0, 0.5, 0.25, 1.0, 0.25, 0.5]","[-, +, +, -, -, +, -, +]","[0.162252, 0.652318, 0.84106, 0.652318, 0.1837...","[-, +, +, -, -, +, -, +]","[0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0]","[=, +, =, -, =, =, =, +]","[1/2, 1, 1, 1/2, 1/2, 1/2, 1/2, 1]","[1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0]","[None, None, None, None, None, None, None, None]","[False, True, False, False, False, False, Fals..."


In [19]:
from sklearn.preprocessing import OneHotEncoder
from fractions import Fraction

def transform_features(df):
    """
    Transforme les features qui ne sont pas des listes de nombres en listes de nombres.

    Transformations spécifiques :
    - 'chromaticinterval': remplace None par 0.
    - 'tonic': encode les valeurs en vecteurs binaires selon 21 possibilités.
    - 'mode': encode les valeurs en vecteurs binaires selon 6 possibilités.
    - 'timesignature': convertit les fractions en nombres décimaux.
    - 'beatfraction': convertit les fractions en nombres décimaux.
    - 'metriccontour', 'imacontour', 'durationcontour': transforme les chaînes en 1, -1 ou 0.
    - 'restduration_frac': convertit les fractions en nombres décimaux, remplace None par 0.0.

    Args:
        df (pd.DataFrame): Le DataFrame d'entrée.

    Returns:
        pd.DataFrame: Le DataFrame transformé.
    """
    if 'chromaticinterval' in df.columns:
        print("chromaticinterval")
        df['chromaticinterval'] = df['chromaticinterval'].apply(
            lambda lst: [0 if val is None else val for val in lst]
        )

    if 'beatstrength' in df.columns:
        print("beatstrength")
        df['beatstrength'] = df['beatstrength'].apply(
            lambda lst: [0 if val is None else val for val in lst]
        )

    if 'beat' in df.columns:
        print("beat")
        df['beat'] = df['beat'].apply(
            lambda lst: [0 if val is None else val for val in lst]
        )

    if 'tonic' in df.columns:
        print("tonic")
        notes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
        alterations = ['-', '', '#']
        possible_tonics = [n + a for n in notes for a in alterations]
        encoder = OneHotEncoder(categories=[possible_tonics], sparse_output=False, handle_unknown='ignore')

        def encode_tonic(tonic_list):
            encoded = encoder.fit_transform(np.array(tonic_list).reshape(-1, 1))
            return encoded.tolist()

        df['tonic'] = df['tonic'].apply(encode_tonic)

    if 'mode' in df.columns:
        print("mode")
        possible_modes = ['major', 'minor', 'dorian', 'phrygian', 'lydian', 'mixolydian']
        encoder = OneHotEncoder(categories=[possible_modes], sparse_output=False, handle_unknown='ignore')

        def encode_mode(mode_list):
            encoded = encoder.fit_transform(np.array(mode_list).reshape(-1, 1))
            return encoded.tolist()

        df['mode'] = df['mode'].apply(encode_mode)

    if 'timesignature' in df.columns:
        print("timesignature")
        df['timesignature'] = df['timesignature'].apply(
            lambda lst: [float(Fraction(val)) if val is not None else 0.0 for val in lst]
        )

    if 'beatfraction' in df.columns:
        print("beatfraction")
        df['beatfraction'] = df['beatfraction'].apply(
            lambda lst: [float(Fraction(val)) if val is not None else 0.0 for val in lst]
        )

    def transform_contour(contour_list):
        return [
            1 if val == '+' else -1 if val == '-' else 0 if val in ('=', None) else val
            for val in contour_list
        ]

    for contour_feature in ['metriccontour', 'imacontour', 'durationcontour']:
        if contour_feature in df.columns:
            print(contour_feature)
            df[contour_feature] = df[contour_feature].apply(transform_contour)

    if 'restduration_frac' in df.columns:
        print("restduration_frac")
        df['restduration_frac'] = df['restduration_frac'].apply(
            lambda lst: [float(Fraction(val)) if val is not None else 0.0 for val in lst]
        )

    if 'phrase_end' in df.columns:
        print("phrase_end")
        df['phrase_end'] = df['phrase_end'].apply(
            lambda lst: 1 if any(val == True for val in lst) else 0
        )

    return df

data = {
    'chromaticinterval': [[1, 2, None, 4], [None, 2, 3, 4]],
    'tonic': [['C#', 'A-', 'B'], ['D', 'E', 'F#']],
    'mode': [['major', 'minor'], ['lydian', 'mixolydian']],
    'timesignature': [['1/4', '1/2', '3/4'], ['2/3', '1/8', '1/16']],
    'beatfraction': [['1/12', '1/3', '5/6'], ['7/8', '1/6', '1/2']],
    'metriccontour': [['+', '-', '='], ['-', '=', None]],
    'imacontour': [['+', '=', '-'], ['=', None, '+']],
    'durationcontour': [[None, '+', '-'], ['-', '+', '=']],
    'restduration_frac': [['1/4', None, '3/8'], [None, '1/16', '1/2']],
    'phrase_end': [[False, False, True], [False, False, False]]
}
df = pd.DataFrame(data)

transformed_df = transform_features(df)

for col in transformed_df.columns:
    print(f"{col}:\n{transformed_df[col][1]}\n")


chromaticinterval
tonic
mode
timesignature
beatfraction
metriccontour
imacontour
durationcontour
restduration_frac
phrase_end
chromaticinterval:
[0, 2, 3, 4]

tonic:
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]]

mode:
[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]

timesignature:
[0.6666666666666666, 0.125, 0.0625]

beatfraction:
[0.875, 0.16666666666666666, 0.5]

metriccontour:
[-1, 0, 0]

imacontour:
[0, 0, 1]

durationcontour:
[-1, 1, 0]

restduration_frac:
[0.0, 0.0625, 0.5]

phrase_end:
0



In [20]:
transformed_sub_seqs_df = transform_features(sub_seqs_df)

chromaticinterval
beatstrength
beat
tonic
mode
timesignature
beatfraction
metriccontour
imacontour
durationcontour
restduration_frac
phrase_end


In [21]:
values = {}
print(transformed_sub_seqs_df.columns)
for col in transformed_sub_seqs_df.columns:
    if col != 'phrase_end':
        print(col)
        for x in transformed_sub_seqs_df[col]:
            if None in x:
                if col not in values:
                    values[col] = 1
                else:
                    values[col] += 1

Index(['midipitch', 'chromaticinterval', 'tonic', 'mode', 'scaledegree',
       'timesignature', 'beatstrength', 'metriccontour', 'imaweight',
       'imacontour', 'duration', 'durationcontour', 'beatfraction', 'beat',
       'restduration_frac', 'phrase_end'],
      dtype='object')
midipitch
chromaticinterval
tonic
mode
scaledegree
timesignature
beatstrength
metriccontour
imaweight
imacontour
duration
durationcontour
beatfraction
beat
restduration_frac


In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Exemple : Fonction de transformation pour des données séquentielles
def transform_sequence_features(df, sequence_columns):
    """
    Transforme des colonnes contenant des listes ou vecteurs en statistiques globales.
    """
    transformed_features = {}
    for col in sequence_columns:
        transformed_features[f'{col}_mean'] = df[col].apply(np.mean)
        transformed_features[f'{col}_std'] = df[col].apply(np.std)
        transformed_features[f'{col}_min'] = df[col].apply(np.min)
        transformed_features[f'{col}_max'] = df[col].apply(np.max)
    return pd.DataFrame(transformed_features)

# Étape 1 : Charger les données
sequence_columns = ['midipitch', 'chromaticinterval', 'scaledegree', 
                    'beatstrength', 'metriccontour', 'duration']
target_column = 'phrase_end'

# Transformer les données
transformed_df = transform_sequence_features(transformed_sub_seqs_df, sequence_columns)
transformed_df['phrase_end'] = transformed_sub_seqs_df[target_column]

# Séparer les données en X et y
X = transformed_df.drop(columns=['phrase_end'])
y = transformed_df['phrase_end']

# Diviser en jeu d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Étape 2 : Tester plusieurs modèles
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'KNN': KNeighborsClassifier(),
}

# Étape 3 : Entraîner et évaluer chaque modèle
for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    print(f"{name}:")
    print(classification_report(y_test, y_pred))
    print("-" * 30)


Logistic Regression:
              precision    recall  f1-score   support

           0       0.64      0.42      0.51      8094
           1       0.75      0.88      0.81     15647

    accuracy                           0.72     23741
   macro avg       0.69      0.65      0.66     23741
weighted avg       0.71      0.72      0.70     23741

------------------------------
SVM:
              precision    recall  f1-score   support

           0       0.71      0.38      0.50      8094
           1       0.74      0.92      0.82     15647

    accuracy                           0.74     23741
   macro avg       0.73      0.65      0.66     23741
weighted avg       0.73      0.74      0.71     23741

------------------------------
Random Forest:
              precision    recall  f1-score   support

           0       0.72      0.55      0.62      8094
           1       0.79      0.89      0.84     15647

    accuracy                           0.77     23741
   macro avg       0.76  

In [23]:
print(models['Random Forest'].score(X_test, y_test))
print(models['Logistic Regression'].score(X_test, y_test))
print(models['SVM'].score(X_test, y_test))
print(models['KNN'].score(X_test, y_test))

/home/gaspar/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.6749083863358747
0.34092919422096796


/home/gaspar/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/gaspar/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


0.6590708057790321


/home/gaspar/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


0.34092919422096796
